# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [137]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
import sqlite3
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [138]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")


In [139]:
# data contract
PerpsV2DataAbi = """
[
    {
      "inputs": [
        {
          "internalType": "contract IAddressResolver",
          "name": "_resolverProxy",
          "type": "address"
        }
      ],
      "payable": false,
      "stateMutability": "nonpayable",
      "type": "constructor"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "allMarketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "allProxiedMarketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "globals",
      "outputs": [
        {
          "components": [
            {
              "internalType": "uint256",
              "name": "minInitialMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationFeeRatio",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationBufferRatio",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "minKeeperFee",
              "type": "uint256"
            }
          ],
          "internalType": "struct PerpsV2MarketData.FuturesGlobals",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "contract IPerpsV2MarketViews",
          "name": "market",
          "type": "address"
        }
      ],
      "name": "marketDetails",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "baseAsset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "marketKey",
              "type": "bytes32"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxLeverage",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "maxMarketValue",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketLimits",
              "name": "limits",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxFundingVelocity",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "skewScale",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FundingParameters",
              "name": "fundingParameters",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "marketSize",
                  "type": "uint256"
                },
                {
                  "components": [
                    {
                      "internalType": "uint256",
                      "name": "long",
                      "type": "uint256"
                    },
                    {
                      "internalType": "uint256",
                      "name": "short",
                      "type": "uint256"
                    }
                  ],
                  "internalType": "struct PerpsV2MarketData.Sides",
                  "name": "sides",
                  "type": "tuple"
                },
                {
                  "internalType": "uint256",
                  "name": "marketDebt",
                  "type": "uint256"
                },
                {
                  "internalType": "int256",
                  "name": "marketSkew",
                  "type": "int256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketSizeDetails",
              "name": "marketSizeDetails",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "price",
                  "type": "uint256"
                },
                {
                  "internalType": "bool",
                  "name": "invalid",
                  "type": "bool"
                }
              ],
              "internalType": "struct PerpsV2MarketData.PriceDetails",
              "name": "priceDetails",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        }
      ],
      "name": "marketDetailsForKey",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "baseAsset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "marketKey",
              "type": "bytes32"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxLeverage",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "maxMarketValue",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketLimits",
              "name": "limits",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxFundingVelocity",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "skewScale",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FundingParameters",
              "name": "fundingParameters",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "marketSize",
                  "type": "uint256"
                },
                {
                  "components": [
                    {
                      "internalType": "uint256",
                      "name": "long",
                      "type": "uint256"
                    },
                    {
                      "internalType": "uint256",
                      "name": "short",
                      "type": "uint256"
                    }
                  ],
                  "internalType": "struct PerpsV2MarketData.Sides",
                  "name": "sides",
                  "type": "tuple"
                },
                {
                  "internalType": "uint256",
                  "name": "marketDebt",
                  "type": "uint256"
                },
                {
                  "internalType": "int256",
                  "name": "marketSkew",
                  "type": "int256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketSizeDetails",
              "name": "marketSizeDetails",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "price",
                  "type": "uint256"
                },
                {
                  "internalType": "bool",
                  "name": "invalid",
                  "type": "bool"
                }
              ],
              "internalType": "struct PerpsV2MarketData.PriceDetails",
              "name": "priceDetails",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "address[]",
          "name": "markets",
          "type": "address[]"
        }
      ],
      "name": "marketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32[]",
          "name": "marketKeys",
          "type": "bytes32[]"
        }
      ],
      "name": "marketSummariesForKeys",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        }
      ],
      "name": "parameters",
      "outputs": [
        {
          "components": [
            {
              "internalType": "uint256",
              "name": "takerFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "overrideCommitFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "takerFeeDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFeeDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "takerFeeOffchainDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFeeOffchainDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxMarketValue",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxFundingVelocity",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "skewScale",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "nextPriceConfirmWindow",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "delayedOrderConfirmWindow",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "minDelayTimeDelta",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxDelayTimeDelta",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "offchainDelayedOrderMinAge",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "offchainDelayedOrderMaxAge",
              "type": "uint256"
            },
            {
              "internalType": "bytes32",
              "name": "offchainMarketKey",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "offchainPriceDivergence",
              "type": "uint256"
            }
          ],
          "internalType": "struct IPerpsV2MarketSettings.Parameters",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "contract IPerpsV2MarketViews",
          "name": "market",
          "type": "address"
        },
        {
          "internalType": "address",
          "name": "account",
          "type": "address"
        }
      ],
      "name": "positionDetails",
      "outputs": [
        {
          "components": [
            {
              "components": [
                {
                  "internalType": "uint64",
                  "name": "id",
                  "type": "uint64"
                },
                {
                  "internalType": "uint64",
                  "name": "lastFundingIndex",
                  "type": "uint64"
                },
                {
                  "internalType": "uint128",
                  "name": "margin",
                  "type": "uint128"
                },
                {
                  "internalType": "uint128",
                  "name": "lastPrice",
                  "type": "uint128"
                },
                {
                  "internalType": "int128",
                  "name": "size",
                  "type": "int128"
                }
              ],
              "internalType": "struct IPerpsV2MarketBaseTypes.Position",
              "name": "position",
              "type": "tuple"
            },
            {
              "internalType": "int256",
              "name": "notionalValue",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "profitLoss",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "accruedFunding",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "remainingMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "accessibleMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationPrice",
              "type": "uint256"
            },
            {
              "internalType": "bool",
              "name": "canLiquidatePosition",
              "type": "bool"
            }
          ],
          "internalType": "struct PerpsV2MarketData.PositionData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        },
        {
          "internalType": "address",
          "name": "account",
          "type": "address"
        }
      ],
      "name": "positionDetailsForMarketKey",
      "outputs": [
        {
          "components": [
            {
              "components": [
                {
                  "internalType": "uint64",
                  "name": "id",
                  "type": "uint64"
                },
                {
                  "internalType": "uint64",
                  "name": "lastFundingIndex",
                  "type": "uint64"
                },
                {
                  "internalType": "uint128",
                  "name": "margin",
                  "type": "uint128"
                },
                {
                  "internalType": "uint128",
                  "name": "lastPrice",
                  "type": "uint128"
                },
                {
                  "internalType": "int128",
                  "name": "size",
                  "type": "int128"
                }
              ],
              "internalType": "struct IPerpsV2MarketBaseTypes.Position",
              "name": "position",
              "type": "tuple"
            },
            {
              "internalType": "int256",
              "name": "notionalValue",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "profitLoss",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "accruedFunding",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "remainingMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "accessibleMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationPrice",
              "type": "uint256"
            },
            {
              "internalType": "bool",
              "name": "canLiquidatePosition",
              "type": "bool"
            }
          ],
          "internalType": "struct PerpsV2MarketData.PositionData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "resolverProxy",
      "outputs": [
        {
          "internalType": "contract IAddressResolver",
          "name": "",
          "type": "address"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    }
  ]
"""

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


## Get transfers and trades

In [140]:
df_trades = pd.read_sql_query('SELECT * FROM trades', con)
df_transfers = pd.read_sql_query('SELECT * FROM transfers', con)


In [141]:
## summarize transfers
df_transfer_summ = df_transfer.groupby('asset')['size'].sum().reset_index()
df_transfer_summ.columns = ['asset', 'netTransfers']

## summarize trades
df_trade_summ = df_trade.groupby('asset')['feesPaid'].sum().reset_index()
df_trade_summ.columns = ['asset', 'feesPaid']


In [142]:
df_trade_summ

,asset,feesPaid
0,AAVE,5075.674714132590485724
1,APE,29895.814346021168908476
2,ATOM,24630.165431171724336227
3,AUD,57.685981547750973994
4,AVAX,41416.994692729125892283
5,AXS,2643.551665373035178878
6,BNB,42580.179727636694019223
7,DOGE,10029.051970208975315282
8,DYDX,34805.992815915972606661
9,EUR,1963.282686577125042571


## Get the market data

In [143]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)
marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call()



In [144]:
markets = [{
    'asset': market[1].decode().replace('\x00', ''),
    'marketDebt': w3.fromWei(market[7], unit='ether')
} for market in marketSummaries]

df_markets = pd.DataFrame(markets)
df_markets

,asset,marketDebt
0,sETH,7806110.763662148515538487
1,sBTC,8737664.121127278390883918
2,LINK,424222.742954252416796321
3,SOL,921070.704145453695034672
4,AVAX,316921.467740602453469963
5,AAVE,1395.904217820525454743
6,UNI,78759.399641961224113488
7,MATIC,494633.777502357860651056
8,APE,56589.558917006258494837
9,DYDX,35082.239127400928953963


In [145]:
df_pnl = df_markets.merge(df_transfer_summ, on='asset').merge(df_trade_summ, on='asset')
df_pnl['netPnl'] = df_pnl['marketDebt'] - df_pnl['netTransfers'] - df_pnl['feesPaid']
df_pnl

,asset,marketDebt,netTransfers,feesPaid,netPnl
0,sETH,7806110.763662148515538487,8723033.098230110605814968,896109.296905756904243084,-1813031.631473718994519565
1,sBTC,8737664.121127278390883918,8676280.354869350340559880,721410.339416689571109028,-660026.573158761520784990
2,LINK,424222.742954252416796321,476978.272313155769741380,83904.441381383055583164,-136659.970740286408528223
3,SOL,921070.704145453695034672,881824.449873798797130093,97808.224901696599677504,-58561.970630041701772925
4,AVAX,316921.467740602453469963,318552.689862562144068246,41416.994692729125892283,-43048.216814688816490566
5,AAVE,1395.904217820525454743,373.546489179793304317,5075.674714132590485724,-4053.316985491858335298
6,UNI,78759.399641961224113488,88736.989326456327251617,10139.086962927789233409,-20116.676647422892371538
7,MATIC,494633.777502357860651056,515181.362830401089127294,76066.650151181403894524,-96614.235479224632370762
8,APE,56589.558917006258494837,26229.054249912356012697,29895.814346021168908476,464.690321072733573664
9,DYDX,35082.239127400928953963,53645.133947317693559428,34805.992815915972606661,-53368.887635832737212126


In [146]:
df_pnl.to_csv('output/market_upnl.csv', index=False)

In [147]:
df_pnl['netPnl'].sum()

Decimal('-3409886.323597287971281758')

In [148]:
df_pnl['netPnlClean'] = df_pnl['netPnl'].apply(lambda x: round(x, 2))
df_pnl[['asset', 'netPnlClean']].sort_values('netPnlClean', ascending=True)

,asset,netPnlClean
0,sETH,-1813031.63
1,sBTC,-660026.57
11,OP,-292967.56
2,LINK,-136659.97
19,FTM,-107779.65
7,MATIC,-96614.24
3,SOL,-58561.97
10,BNB,-53741.81
9,DYDX,-53368.89
4,AVAX,-43048.22
